## The Observer pattern

The Observer pattern is useful for state monitoring and event handling situations.
This pattern allows a given object to be monitored by an unknown and dynamic
group of observer objects. The core object being observed needs to implement an
interface that makes it observable.
Whenever a value on the core object changes, it lets all the observer objects know
that a change has occurred, by calling a method announcing there's been a change
of state.

![](uml/observer_pattern.png)

### An Observer example

In [1]:
from __future__ import annotations
from typing import Protocol, List, Type
from datetime import datetime
import abc
import time
import random


class Die(abc.ABC):
    def __init__(self) -> None:
        self.face: int
        self.roll()
        
    @abc.abstractmethod
    def roll(self) -> None: ...
        
    def __repr__(self) -> str:
        return f"{self.face}"
    
    
class D6(Die):
    def roll(self) -> None:
        self.face = random.randint(1, 6)
    
class Dice(abc.ABC):
    def __init__(self, n: int, die_class: Type[Die]) -> None:
        self.dice = [die_class() for _ in range(n)]
        
    @abc.abstractmethod
    def roll(self) -> None: ...
        
    @property
    def total(self) -> int:
        return sum(d.face for d in self.dice)
    
    
class SimpleDice(Dice):
    def roll(self) -> None:
        for d in self.dice:
            d.roll()
            

class Observable:
    def __init__(self) -> None:
        self._observers: list[Observer] = []
            
    def attach(self, observer: Observer) -> None:
        self._observers.append(observer)
            
    def detach(self, observer: Observer) -> None:
        self._observers.remove(observer)
        
    def _notify_observers(self) -> None:
        for observer in self._observers:
            observer()
            
        
Hand = List[Die]

class ZonkHandHistory(Observable):
    def __init__(self, player: str, dice_set: Dice) -> None:
        super().__init__()
        self.player = player
        self.dice_set = dice_set
        self.rolls: list[Hand]
            
    def start(self) -> Hand:
        self.dice_set.roll()
        self.rolls = [self.dice_set.dice]
        self._notify_observers()  # State change
        return self.dice_set.dice
    
    def roll(self) -> Hand:
        self.dice_set.roll()
        self.rolls.append(self.dice_set.dice)
        self._notify_observers()  # State change
        return self.dice_set.dice
        

class Observer(Protocol):
    def __call__(self) -> None:
        ...
                
            
class SaveZonkHand(Observer):
    def __init__(self, hand: ZonkHandHistory) -> None:
        self.hand = hand
        self.count = 0
        
    def __call__(self) -> None:
        self.count += 1
        message = {
            "player": self.hand.player,
            "sequence": self.count,
            "hands": str(self.hand.rolls),
            "time": datetime.now().strftime("%H:%M:%S"),
        }
        print(f"SaveZonkHand {message}")
        

class ThreeDistinctValues(Observer):
    """Observer of ZonkHandHistory"""
    def __init__(self, hand: ZonkHandHistory) -> None:
        self.hand = hand
        
    def __call__(self) -> None:
        last_roll = self.hand.rolls[-1]
        distinct_values = set(last_roll)
        print(distinct_values)
        if len(distinct_values) >= 3:
            print('3 Distinct!')
        
        

sd = SimpleDice(6, D6)  # Make a dice
player = ZonkHandHistory("Bo", sd)  # Make a player 
save_history = SaveZonkHand(player)  # Make an observer 1
three_distinct = ThreeDistinctValues(player)  # Make an observer 2
player.attach(save_history)  # Attach an observer 1
player.attach(three_distinct)  # Attach an observer 2
r1 = player.start()  # Start
print(r1, end='\n\n')
time.sleep(1)
player.detach(save_history)  # Detach an observer 1
r2 = player.roll()

SaveZonkHand {'player': 'Bo', 'sequence': 1, 'hands': '[[1, 5, 6, 1, 1, 5]]', 'time': '12:16:52'}
{5, 1, 5, 6, 1, 1}
3 Distinct!
[1, 5, 6, 1, 1, 5]

{6, 1, 1, 5, 1, 1}
3 Distinct!


In [2]:
if __name__ == '__main__':        
    import doctest
    import subprocess
    name = "03-The Observer pattern"
    doctest.testmod(verbose=False)
    subprocess.run(f'jupyter nbconvert --to script --output test "{name}"', shell=True)
    std_out = subprocess.run('mypy --strict test.py', capture_output=True, shell=True).stdout
    print(std_out.decode('ascii'))

[NbConvertApp] Converting notebook 03-The Observer pattern.ipynb to script
[NbConvertApp] Writing 4373 bytes to test.py


Success: no issues found in 1 source file

